In [1]:
%reload_ext tensorboard


# import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix
# from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report


In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Convolution2D, Conv1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling1D
from keras import backend as K
from keras import backend
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import time
import os
import psutil
import csv
from itertools import repeat
from PIL import Image
from numpy import asarray
from scipy.stats import entropy
import math

In [3]:
# algoName='CNN'
# activationFun='relu'
# optimizerName='Adam'
# def createDeepModel():
#     model = Sequential()
    
#     if(algoName=='CNN'):    
#         model.add(Conv1D(filters=5, kernel_size=3, activation=activationFun,input_shape = (x_shape, 1))) # changed x_data.shape[1] to x_shape
#         model.add(Dropout(0.1))
#         model.add(MaxPooling1D(pool_size=3))
        
#         model.add(Conv1D(filters=5, kernel_size=3, activation=activationFun))
#         model.add(Dropout(0.05))
#         model.add(MaxPooling1D(pool_size=3))
        
#         model.add(Flatten())  # Uncommented this line
#         model.add(Dense(x_shape, activation=activationFun))
#         model.add(Dense(mean_shape, activation=activationFun))
#         model.add(Dense(50, activation=activationFun))
#         model.add(Dense(outputClasses, activation='sigmoid'))
#         model.compile(loss='mean_squared_error', optimizer=optimizerName, metrics=['accuracy'])

#     return model
# deepModel=createDeepModel()

In [4]:
accList, precList, recallList, f1List = [], [], [], []
numOfClients=4
numOfIterations=1
deepModelAggWeights=[]
firstClientFlag=True
modelLocation='C:\\Users\\PRAKHAR\\Documents\\GitHub\\dsci2023\\saved_model\\'+"final_model.h5"
# Load the global model from the .h5 file
globalModel = load_model('final_model.h5')
globalModelFinal = load_model('final_model.h5')
def updateServerModel(clientModel, clientModelWeight) :
    global firstClientFlag
    for ind in range(len(clientModelWeight)):
        if(firstClientFlag==True):
            deepModelAggWeights.append(clientModelWeight[ind])   
        else:
            deepModelAggWeights[ind]=(deepModelAggWeights[ind]+clientModelWeight[ind])
            
def updateClientsModels():
    global clientsModelList
    clientsModelList.clear()
    for clientID in range(numOfClients):
        m = keras.models.clone_model(globalModel)
        m.set_weights(globalModel.get_weights())
        clientsModelList.append(m)

In [5]:
import time
import os
import psutil
import numpy as np
import pandas as pd
from scipy.spatial import distance
from keras.models import load_model

start_time = time.time()
process = psutil.Process(os.getpid())
arr = []
cols, rows = numOfClients, numOfIterations

def flatten_weights(weights):
    flattened_weights = []
    for weight in weights:
        flattened_weights.extend(weight.flatten())
    return np.array(flattened_weights)

# def compute_euclidean_distance(weights1, weights2):
#     return distance.euclidean(weights1, weights2)
# def compute_hausdorff_distance(set1, set2):
#     max_distance = 0.0
#     for point1 in set1:
#         min_distance = float('inf')
#         for point2 in set2:
#             distance = compute_euclidean_distance(point1, point2)  # Replace 'some_distance_function' with your distance function d(a, b)
#             min_distance = min(min_distance, distance)
#         max_distance = max(max_distance, min_distance)
#     return max_distance
def compute_hausdorff_distance(set1, set2):
    # Reshape the sets to have a shape of (N, 1, D) for broadcasting
    set1 = set1.reshape(-1, 1, set1.shape[-1])
    set2 = set2.reshape(1, -1, set2.shape[-1])
    
    # Compute the pairwise Euclidean distances between points in set1 and set2
    distances = np.linalg.norm(set1 - set2, axis=-1)
    # Find the maximum distance from set1 to set2 and vice versa
    max_to_set2 = np.max(np.min(distances, axis=1))
    max_to_set1 = np.max(np.min(distances, axis=0))
    # Return the Hausdorff distance
    return max(max_to_set1, max_to_set2)
# Function to calculate the cosine similarity between two vectors
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

# Function to calculate the cosine difference between two vectors
def cosine_difference(a, b):
    return 1 - cosine_similarity(a, b)
# Get the weights of the global model
global_weights = flatten_weights(globalModel.get_weights())
print(len(global_weights))

# Load client models from .h5 files and store them in a list
clientsModelList = []
for clientID in range(numOfClients):
    client_model = load_model(f"client_model_{clientID}.h5")
    clientsModelList.append(client_model)
print(len(clientsModelList))


2156
4


In [6]:
for iterationNo in range(1,numOfIterations+1):
    print("Iteration",iterationNo)
#     dis = []
    for clientID in range(numOfClients):
        print("clientID",clientID)
        clientWeight=clientsModelList[clientID].get_weights() 
        # Compute and print Euclidean distance
        client_weights = flatten_weights(clientWeight)
        print(client_weights)
#         hausdorff_distance = compute_hausdorff_distance(global_weights, client_weights)
#         print(f"Hausdorff distance between global model and client {clientID}: {hausdorff_distance}")
#         dis.append(hausdorff_distance)
        updateServerModel(clientsModelList[clientID], clientWeight)
        firstClientFlag=False
#     arr.append(dis)
    #Average all clients model
    print(len(deepModelAggWeights))
    for ind in range(len(deepModelAggWeights)):
        deepModelAggWeights[ind]/=numOfClients
#         print(deepModelAggWeights[ind])
    dw_last=globalModel.get_weights()
    
    print(len(dw_last))
    for ind in range(len(deepModelAggWeights)): 
        dw_last[ind]=deepModelAggWeights[ind]
    
    #Update server's model
    globalModel.set_weights(dw_last)
    dw_last = deepModelAggWeights.copy()
    globalModel.set_weights(dw_last)
    print(flatten_weights(globalModel.get_weights()))
    print("Server's model updated")
    print("Saving model . . .")
    globalModel.save("final_model.h5")
    # Servers model is updated, now it can be used again by the clients
#     updateClientsModels()
    firstClientFlag=True
    deepModelAggWeights.clear()

Iteration 1
clientID 0
[-0.2906119   0.50184655 -0.53207994 ...  0.09069712  0.03928225
 -0.03707767]
clientID 1
[-0.395617    0.48739505 -0.48584825 ...  0.15840667 -0.07099604
  0.07232244]
clientID 2
[-0.37091103  0.53774303 -0.5352819  ...  0.1641018  -0.05915951
  0.06118244]
clientID 3
[-0.37881097  0.5400526  -0.5193805  ...  0.16242798 -0.10011385
  0.10700398]
12
12
[-0.35898772  0.51675934 -0.51814765 ...  0.14390838 -0.04774679
  0.0508578 ]
Server's model updated
Saving model . . .


In [7]:
# Step 1: Calculate the difference between the updated global model and globalModelFinal
global_model_difference = []
global_model_final_weights = flatten_weights(globalModelFinal.get_weights())
global_model_weights = flatten_weights(globalModel.get_weights())

for i in range(len(global_model_final_weights)):
    diff = global_model_weights[i] - global_model_final_weights[i]
    global_model_difference.append(diff)
global_model_difference = flatten_weights(global_model_difference)

# Step 2: Compute the Hausdorff distance between global_model_difference and each client model
hausdorff_distances = []
cosine_differences = []
for clientID in range(numOfClients):
    client_weights = flatten_weights(clientsModelList[clientID].get_weights())
    distance = compute_hausdorff_distance(global_model_difference, client_weights)
    hausdorff_distances.append(distance)
    cosine_diff = cosine_difference(global_model_difference, client_weights)
    cosine_differences.append(cosine_diff)
print(len(client_weights))
# Print the computed Hausdorff distances
for clientID, (distance, cosine_diff) in enumerate(zip(hausdorff_distances, cosine_differences)):
    print(f"Client {clientID} Hausdorff Distance: {distance}, Cosine Difference: {cosine_diff}")

2156
Client 0 Hausdorff Distance: 11.46939754486084, Cosine Difference: nan
Client 1 Hausdorff Distance: 11.394619941711426, Cosine Difference: nan
Client 2 Hausdorff Distance: 11.244811058044434, Cosine Difference: nan
Client 3 Hausdorff Distance: 11.412675857543945, Cosine Difference: nan


C:\Users\PRAKHAR\AppData\Local\Temp\ipykernel_17536\864390595.py:50: RuntimeWarning: invalid value encountered in float_scalars
  return dot_product / (norm_a * norm_b)


2


ValueError: You called `set_weights(weights)` on layer "sequential" with a weight list of length 2156, but the layer was expecting 12 weights. Provided weights: [-0.35103885  0.50277019 -0.50672483 ...  0.130446...

In [ ]:
global_weights = flatten_weights(globalModel.get_weights())
print(global_weights)
global_final_weights = flatten_weights(globalModelFinal.get_weights())
print(global_final_weights)